<a href="https://colab.research.google.com/github/xjtulyc/lyc_code/blob/master/VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VGG16 Tensorflow实现


## VGG16 神经网络
VGG，又叫VGG-16，顾名思义就是有16层，包括13个卷积层和3个全连接层，是由Visual Geometry Group组的Simonyan和Zisserman在文献《Very Deep Convolutional Networks for Large Scale Image Recognition》中提出卷积神经网络模型，该模型主要工作是证明了增加网络的深度能够在一定程度上影响网络最终的性能。其年参加了ImageNet图像分类与定位挑战赛，取得了在分类任务上排名第二，在定位任务上排名第一的优异成绩。



## 结构
根据卷积核大小与卷积层数目不同，VGG可以分为6种子模型，分别是A、A-LRN、B、C、D、E，我们常看到的基本是D、E这两种模型，让我们看下面官方给出的6种结构图：


<img src='https://img-blog.csdnimg.cn/20210310095311914.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3FxXzQ1OTk4MDQx,size_16,color_FFFFFF,t_70#pic_center'>



以配置D为例，在卷积层1（conv3-64）,卷积层2（conv3-128）,卷积层3（conv3-256）,卷积层4（conv3-512）分别有64个，128个，256个，512个3X3卷积核，在每两层之间有池化层为移动步长为2的2X2池化矩阵（maxpool）。在卷积层5（conv3-512）后有全连接层，再之后是soft-max预测层。


这张图可能会更好地理解


<img src='https://img-blog.csdnimg.cn/20210310115556457.png#pic_center'>


图像张量尺寸变化：224->112->56->28->14->7


## 数据与训练

1. 输入处理

如果要使用224x224的图作为训练的输入，用S表示图片最小边的值，当S=224时这个图就直接使用，直接将多余的部分减掉；对于S远大于224的，模型将对图像进行单尺度和多尺度的剪裁，通过剪切这个图片中包含object的子图作为训练数据。

2. 训练方式

采用带动量的小批量梯度下降法，来优化目标函数，并且当学习效果较为满意时，最初加入的学习率权重衰减系数会起作用，会减小学习率，缓慢达到最优解。

3. 初始化

若为浅层，则先随机初始化后训练，深层采用浅层训练后的数据作为初始化数据，中间层则随机初始化。


## 特色

1. 卷积层均采用相同的卷积核参数，这样就能够使得每一个卷积层（张量）与前一层（张量）保持相同的宽和高；

2. 池化层均采用相同的池化核参数，池化层采用尺寸为2X2，stride=2，max的池化方式，使得池化后的层尺寸长宽为未池化前的1/2；

3. 利用小尺寸卷积核等效大尺寸卷积核，2个3X3卷积核的感受野与1个5X5卷积核相当，3个3X3卷积核与1个7X7卷积核相当，故在特征提取效果相当时，多个小卷核与大卷积核相比，学习参数更少，计算量较小，训练更加快速，还能增加网络的深度，提升模型性能。

下面用计算说明为什么2个3X3卷积核能代替1个5X5卷积核：

<img src='https://img-blog.csdnimg.cn/20210310111310615.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3FxXzQ1OTk4MDQx,size_16,color_FFFFFF,t_70#pic_center'>

假设feature map是28×28的，假设卷记的步长step=1，padding=0：

使用一层5X5卷积核

由(28−5)/1+1=24可得，输出的feature map为24X24
使用两层3X3卷积核

第一层，由(28−3)/1+1=26可得，输出的feature map为26X26

第二层，由(26−3)/1+1=24可得，输出的feature map为24X24

故最终尺寸结果相等，用3X3卷积核代替7X7卷积核同理。


## 实验结果

### 单尺度评估
<img src=''>

In [1]:
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras import Model

In [2]:
np.set_printoptions(threshold=np.inf)

dataset = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = dataset.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [4]:
class VGG16(Model):
    def __init__(self):
        super(VGG16, self).__init__()
        self.c1 = Conv2D(filters=64, kernel_size=(3, 3), padding='same')  # 卷积层1
        self.b1 = BatchNormalization()  # BN层1
        self.a1 = Activation('relu')  # 激活层1
        self.c2 = Conv2D(filters=64, kernel_size=(3, 3), padding='same', )
        self.b2 = BatchNormalization()  # BN层1
        self.a2 = Activation('relu')  # 激活层1
        self.p1 = MaxPool2D(pool_size=(2, 2), strides=2, padding='same')
        self.d1 = Dropout(0.2)  # dropout层

        self.c3 = Conv2D(filters=128, kernel_size=(3, 3), padding='same')
        self.b3 = BatchNormalization()  # BN层1
        self.a3 = Activation('relu')  # 激活层1
        self.c4 = Conv2D(filters=128, kernel_size=(3, 3), padding='same')
        self.b4 = BatchNormalization()  # BN层1
        self.a4 = Activation('relu')  # 激活层1
        self.p2 = MaxPool2D(pool_size=(2, 2), strides=2, padding='same')
        self.d2 = Dropout(0.2)  # dropout层

        self.c5 = Conv2D(filters=256, kernel_size=(3, 3), padding='same')
        self.b5 = BatchNormalization()  # BN层1
        self.a5 = Activation('relu')  # 激活层1
        self.c6 = Conv2D(filters=256, kernel_size=(3, 3), padding='same')
        self.b6 = BatchNormalization()  # BN层1
        self.a6 = Activation('relu')  # 激活层1
        self.c7 = Conv2D(filters=256, kernel_size=(3, 3), padding='same')
        self.b7 = BatchNormalization()
        self.a7 = Activation('relu')
        self.p3 = MaxPool2D(pool_size=(2, 2), strides=2, padding='same')
        self.d3 = Dropout(0.2)

        self.c8 = Conv2D(filters=512, kernel_size=(3, 3), padding='same')
        self.b8 = BatchNormalization()  # BN层1
        self.a8 = Activation('relu')  # 激活层1
        self.c9 = Conv2D(filters=512, kernel_size=(3, 3), padding='same')
        self.b9 = BatchNormalization()  # BN层1
        self.a9 = Activation('relu')  # 激活层1
        self.c10 = Conv2D(filters=512, kernel_size=(3, 3), padding='same')
        self.b10 = BatchNormalization()
        self.a10 = Activation('relu')
        self.p4 = MaxPool2D(pool_size=(2, 2), strides=2, padding='same')
        self.d4 = Dropout(0.2)

        self.c11 = Conv2D(filters=512, kernel_size=(3, 3), padding='same')
        self.b11 = BatchNormalization()  # BN层1
        self.a11 = Activation('relu')  # 激活层1
        self.c12 = Conv2D(filters=512, kernel_size=(3, 3), padding='same')
        self.b12 = BatchNormalization()  # BN层1
        self.a12 = Activation('relu')  # 激活层1
        self.c13 = Conv2D(filters=512, kernel_size=(3, 3), padding='same')
        self.b13 = BatchNormalization()
        self.a13 = Activation('relu')
        self.p5 = MaxPool2D(pool_size=(2, 2), strides=2, padding='same')
        self.d5 = Dropout(0.2)

        self.flatten = Flatten()
        self.f1 = Dense(512, activation='relu')
        self.d6 = Dropout(0.2)
        self.f2 = Dense(512, activation='relu')
        self.d7 = Dropout(0.2)
        self.f3 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.c1(x)
        x = self.b1(x)
        x = self.a1(x)
        x = self.c2(x)
        x = self.b2(x)
        x = self.a2(x)
        x = self.p1(x)
        x = self.d1(x)

        x = self.c3(x)
        x = self.b3(x)
        x = self.a3(x)
        x = self.c4(x)
        x = self.b4(x)
        x = self.a4(x)
        x = self.p2(x)
        x = self.d2(x)

        x = self.c5(x)
        x = self.b5(x)
        x = self.a5(x)
        x = self.c6(x)
        x = self.b6(x)
        x = self.a6(x)
        x = self.c7(x)
        x = self.b7(x)
        x = self.a7(x)
        x = self.p3(x)
        x = self.d3(x)

        x = self.c8(x)
        x = self.b8(x)
        x = self.a8(x)
        x = self.c9(x)
        x = self.b9(x)
        x = self.a9(x)
        x = self.c10(x)
        x = self.b10(x)
        x = self.a10(x)
        x = self.p4(x)
        x = self.d4(x)

        x = self.c11(x)
        x = self.b11(x)
        x = self.a11(x)
        x = self.c12(x)
        x = self.b12(x)
        x = self.a12(x)
        x = self.c13(x)
        x = self.b13(x)
        x = self.a13(x)
        x = self.p5(x)
        x = self.d5(x)

        x = self.flatten(x)
        x = self.f1(x)
        x = self.d6(x)
        x = self.f2(x)
        x = self.d7(x)
        y = self.f3(x)
        return y

In [5]:
with tf.device('/gpu:0'):
  model = VGG16()

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                metrics=['sparse_categorical_accuracy'])

  checkpoint_save_path = "./checkpoint/VGG16.ckpt"
  if os.path.exists(checkpoint_save_path + '.index'):
      print('-------------load the model-----------------')
      model.load_weights(checkpoint_save_path)

  cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
                                                  save_weights_only=True,
                                                  save_best_only=True)

  history = model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_test, y_test), validation_freq=1,
                      callbacks=[cp_callback])
  model.summary()

  # print(model.trainable_variables)
  file = open('./weights.txt', 'w')
  for v in model.trainable_variables:
      file.write(str(v.name) + '\n')
      file.write(str(v.shape) + '\n')
      file.write(str(v.numpy()) + '\n')
  file.close()

Epoch 1/5


ValueError: ignored

In [ ]:
###############################################    show   ###############################################

# 显示训练集和验证集的acc和loss曲线
acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()